## This notebook serves as an easy, interactive querying method for calling  C-GRASP East Coast, US, sediment data!

### To start off, run this first cell, select the dataset that you are interested in, and enter in the years in which you want to call data from

### Select which sediment dataset you would like to access from the C-Grasp Zenodo site (see more at https://zenodo.org/record/5874231/#.YetG3NuIajg). Simply click on what ever dataset you prefer. These dataset so far are:
  * Entire Coastal Dataset: This is all data that is found to be within 10km of the Natural Earth coastline polyline
  * Estimated Onshore Dataset: This is all the data from "Entire Coastal Dataset" that lies within the Natural Earth United States Polygon
  * Verified Onshore Dataset: This is all data that was able to be verified onshore from either sampling method, note, or location type data

### Enter in the beginning year that you want to query all of your data from.

### Enter the end year of dates that you want to query samples from using the sample method from "Start Year".

In [ ]:
import ipyleaflet
import ogr, os
import pandas as pd
import geopandas as gpd
import shapefile
import json 
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl, basemaps, basemap_to_tiles, GeoData
)

import pyproj
from shapely.ops import transform
from shapely.geometry import Polygon
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import ipywidgets
import osmnx as ox
from shapely.geometry import mapping, shape
from shapely.geometry import box as shBox
import numpy as np
import urllib.request


#Dataset collection widget
zen=ipywidgets.Select(
    options=['Entire Coastal Dataset', 'Estimated Onshore Dataset', 'Verified Onshore Dataset'],
    value='Entire Coastal Dataset',
    # rows=10,
    description='Dataset:',
    disabled=False
)

display(zen)


print('Enter Year Range of Interest Interest Below:')

#Lower bound year text enter widget

y0=ipywidgets.Text(
    value='yyyy',
    placeholder='Type something',
    description='Start Year:',
    disabled=False
)

display(y0)

#Upper bound year text enter widget
y1=ipywidgets.Text(
    value='yyyy',
    placeholder='Type something',
    description='End Year:',
    disabled=False
)

display(y1)



### After entering the requested data above, run this next cell to further query your data

#### Your data will be called from Zenodo to your system. This will take varying amounts of time depending on how much data you query and which dataset you choose.

#### After the data is downloaded, a map will appear where you spatially query your desired data (the points are all available data). See the Readme.md file for more information on how to use this map.

#### Under the map is a frequency histogram of the years in which samples are available from. 

In [ ]:
if float(y1.value)-float(y0.value)<0: #Making sure your date range is appropriate
    print('Error! End Date Proceeds Start Date!')

#Call the chosen dataset from Zenodo
else:
    url = 'https://zenodo.org/record/5874231/files/' 
    if zen.value=='Entire Coastal Dataset':
        filename='dataset_10kmcoast.csv'
    if zen.value=='Estimated Onshore Dataset':
        filename='Data_EstimatedOnshore.csv'
    if zen.value=='Verified Onshore Dataset':
        filename='Data_VerifiedOnshore.csv'

    url=(url+filename)
    print('Retrieving Data, Please Wait')
    #retrieve data
    df=pd.read_csv(url)
    print('Data Retrieved! Now Subsetting to Year')
    df['Date'] = pd.to_datetime(df['Date']) #Convert sample date to pandas datetime object
    df['year'] = df['Date'].dt.year #Extract year from datetime object
    df=df.loc[df.year>=float(y0.value), :]     #subset by lower year
    df=df.loc[df.year<=float(y1.value), :]  #subset by upper year

    df = pd.DataFrame(df.drop(columns='year'))

    if float(y1.value)-float(y0.value)>0:
        print()
        print('Done. '+str(len(df))+' samples available from dates below map')
        df=df[df['Date'].astype("datetime64").dt.year<2060] #remove any data with typos from source data
        #print statistics
        df['Mean'].groupby(df['Date'].astype("datetime64").dt.year).count().plot(kind="bar",figsize=(20,3))
        print()
        print('Draw Spatial Query Below')
    else:
        print('Done. '+str(len(df))+' samples available')
        print('Draw Spatial Query Below')

#make map with geodataframe
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude)) #convert dataframe to geodataframe
gdf['Date'] = gdf['Date'].astype(str)

gdf= GeoData(geo_dataframe = gdf)
watercolor = basemap_to_tiles(basemaps.Esri.WorldImagery, crs='espg:4326')

m = Map(layers=(watercolor, ), center=(35,-75 ), zoom=4)

m.add_layer(gdf)


#Add draw tool

dc = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}},
                 rectangle={'shapeOptions': {'color': '#0000FF'}},
                 circle={'shapeOptions': {'color': '#0000FF'}},
                 circlemarker={},
                 )
def handle_draw(target, action, geo_json):
    print(action)
    print(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)
    

    
m

### Run the next cell to download your data to your system

#### The cell will show a frequency histogram for the year dates of the samples downloaded to your computer. If you would like to alter your query, select the "restart the kernel" button to your Jupyter notebook tool bar and repeat the above tasks. 


In [ ]:
#This extracts the polygon drawn in the above map
bounds= (dc.last_draw).get("geometry").get("coordinates")[0]
bounds_poly=Polygon(bounds)
bounds_gpd=gpd.GeoDataFrame(geometry=[bounds_poly])
polygon=bounds_gpd.set_crs('EPSG:4326')
#These lines turn your dataset into a geodataframe and extract points within the polygon
df=gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
df=df.set_crs('EPSG:4326')
df=gpd.clip(df, polygon)
df=pd.DataFrame(df)
#Download to file
df.to_csv('../data.csv')
#Print date statistics if more than one year of data
if float(y1.value)-float(y0.value)>0:
    print(str(len(df))+' samples downloaded from dates:')
    df['Mean'].groupby(df['Date'].astype("datetime64").dt.year).count().plot(kind="bar",figsize=(20,3))
#Print length statistics if one year of data
else:
    print(str(len(df)) +'samples downloaded')

